In [27]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import  time 
from torch.autograd import Variable
from torch.utils.data import DataLoader
from skimage.metrics import structural_similarity as compare_ssim
from torch.nn.modules.loss import _Loss 
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *
from torchvision.utils import save_image

In [28]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [30]:
def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [31]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

In [32]:

generator = Generator().cuda()
generator.load_state_dict(torch.load("./saved_models_uils/G/generator_800.pth"))

FileNotFoundError: [Errno 2] No such file or directory: './saved_models_uils/G/generator_800.pth'

In [ ]:
generator.eval()

In [ ]:

path= 'F:/Test-Output/ComparisonResults/input/input/'#要改
path_list = os.listdir(path)
# print(path_list)
path_list.sort(key=lambda x:int(x.split('.')[0]))


pathd= 'F:/Test-Output/ComparisonResults/input/U90-D/'#要改
pathd_list = os.listdir(pathd)
# print(pathd_list)
pathd_list.sort(key=lambda x:int(x.split('.')[0]))

i=1
for item,itemd in zip(path_list ,pathd_list):
    
    impath=path+item
    impathd = pathd + itemd
    # print(impath)
    imgx= cv2.imread(path+item)
    imgx= cv2.imread(impath)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)
    
    imgd= cv2.imread(pathd+itemd)
    # imgx= cv2.imread(impath)
    imgd=cv2.resize(imgd,(256,256))
    imgd = cv2.cvtColor(imgd, cv2.COLOR_BGR2RGB)
    imgd = np.array(imgd).astype(dtype)

    imgx= torch.from_numpy(imgx)
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx).cuda()
    #print(imgx.shape)
    
    imgd= torch.from_numpy(imgd)
    imgd=imgd.permute(2,0,1).unsqueeze(0)
    imgd=imgd/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgd = Variable(imgd).cuda()
    #print(imgx.shape)
    
    output=generator(imgx,imgd)
    out=output[3].data

    save_image(out,'F:/Test-Output/ComparisonResults/input/output/'+item, nrow=5, normalize=True)
    i=i+1


In [ ]:
def compute_psnr(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def compute_mse(img1,img2):
    mse=np.mean( (img1/255. - img2/255.) ** 2 )
    return mse



In [ ]:
from scipy import ndimage
from PIL import Image
import numpy as np
import math

In [ ]:


def _uiconm(x, window_size):

    plip_lambda = 1026.0
    plip_gamma  = 1026.0
    plip_beta   = 1.0
    plip_mu     = 1026.0
    plip_k      = 1026.0
    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1]/window_size
    k2 = x.shape[0]/window_size
    # weight
    w = -1./(k1*k2)
    blocksize_x = window_size
    blocksize_y = window_size
    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[0:int(blocksize_y*k2), 0:int(blocksize_x*k1)]
    # entropy scale - higher helps with randomness
    alpha = 1
    val = 0
    k1 = int(k1)
    k2 = int(k2)
    for l in range(k1):
        for k in range(k2):
            block = x[k*window_size:window_size*(k+1), l*window_size:window_size*(l+1), :]
            max_ = np.max(block)
            min_ = np.min(block)
            top = max_-min_
            bot = max_+min_
            if math.isnan(top) or math.isnan(bot) or bot == 0.0 or top == 0.0: val += 0.0
            else: val += alpha*math.pow((top/bot),alpha) * math.log(top/bot)
            #try: val += plip_multiplication((top/bot),math.log(top/bot))
    return w*val

def mu_a(x, alpha_L=0.1, alpha_R=0.1):
    """
      Calculates the asymetric alpha-trimmed mean
    """
    # sort pixels by intensity - for clipping
    x = sorted(x)
    # get number of pixels
    K = len(x)
    # calculate T alpha L and T alpha R
    T_a_L = math.ceil(alpha_L*K)
    T_a_R = math.floor(alpha_R*K)
    # calculate mu_alpha weight
    weight = (1/(K-T_a_L-T_a_R))
    # loop through flattened image starting at T_a_L+1 and ending at K-T_a_R
    s   = int(T_a_L+1)
    e   = int(K-T_a_R)
    val = sum(x[s:e])
    val = weight*val
    return val

def s_a(x, mu):
    val = 0
    for pixel in x:
        val += math.pow((pixel-mu), 2)
    return val/len(x)


def _uicm(x):
    R = x[:,:,0].flatten()
    G = x[:,:,1].flatten()
    B = x[:,:,2].flatten()
    RG = R-G
    YB = ((R+G)/2)-B
    mu_a_RG = mu_a(RG)
    mu_a_YB = mu_a(YB)
    s_a_RG = s_a(RG, mu_a_RG)
    s_a_YB = s_a(YB, mu_a_YB)
    l = math.sqrt( (math.pow(mu_a_RG,2)+math.pow(mu_a_YB,2)) )
    r = math.sqrt(s_a_RG+s_a_YB)
    return (-0.0268*l)+(0.1586*r)


def sobel(x):
    dx = ndimage.sobel(x,0)
    dy = ndimage.sobel(x,1)
    mag = np.hypot(dx, dy)
    mag *= 255.0 / np.max(mag) 
    return mag

def _uism(x):
    """
      Underwater Image Sharpness Measure
    """
    # get image channels
    R = x[:,:,0]
    G = x[:,:,1]
    B = x[:,:,2]
    # first apply Sobel edge detector to each RGB component
    Rs = sobel(R)
    Gs = sobel(G)
    Bs = sobel(B)
    # multiply the edges detected for each channel by the channel itself
    R_edge_map = np.multiply(Rs, R)
    G_edge_map = np.multiply(Gs, G)
    B_edge_map = np.multiply(Bs, B)
    # get eme for each channel
    r_eme = eme(R_edge_map, 10)
    g_eme = eme(G_edge_map, 10)
    b_eme = eme(B_edge_map, 10)
    # coefficients
    lambda_r = 0.299
    lambda_g = 0.587
    lambda_b = 0.144
    return (lambda_r*r_eme) + (lambda_g*g_eme) + (lambda_b*b_eme)


def eme(x, window_size):
    """
      Enhancement measure estimation
      x.shape[0] = height
      x.shape[1] = width
    """
    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1]/window_size
    k2 = x.shape[0]/window_size
    # weight
    w = 2./(k1*k2)
    blocksize_x = window_size
    blocksize_y = window_size
    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[0:int(blocksize_y*k2), 0:int(blocksize_x*k1)]
    val = 0
    k1 = int(k1)
    k2 = int(k2)
    for l in range(k1):
        for k in range(k2):
            block = x[k*window_size:window_size*(k+1), l*window_size:window_size*(l+1)]
            max_ = np.max(block)
            min_ = np.min(block)
            # bound checks, can't do log(0)
            if min_ == 0.0: val += 0
            elif max_ == 0.0: val += 0
            else: val += math.log(max_/min_)
    return w*val



def calculate_uiqm(x):
    """
      Function to return UIQM to be called from other programs
      x: image
    """
    x = x.astype(np.float32)

    c1 = 0.0282; c2 = 0.2953; c3 = 3.5753
    uicm   = _uicm(x)
    uism   = _uism(x)
    uiconm = _uiconm(x, 10)
    uiqm = (c1*uicm) + (c2*uism) + (c3*uiconm)
    return uiqm

In [ ]:

import cv2


In [ ]:


def getUCIQE(img):
    img_BGR = img
    img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB) 
    img_LAB = np.array(img_LAB,dtype=np.float64)
    # Trained coefficients are c1=0.4680, c2=0.2745, c3=0.2576 according to paper.
    coe_Metric = [0.4680, 0.2745, 0.2576]
    
    # img_lum = img_LAB[:,:,0]/255.0
    # img_a = img_LAB[:,:,1]/255.0
    # img_b = img_LAB[:,:,2]/255.0
    
    img_lum = img_LAB[:,:,0] / 100.0
    img_a = (img_LAB[:,:,1] - 128) / 127.0
    img_b = (img_LAB[:,:,2] - 128) / 127.0

    # item-1
    chroma = np.sqrt(np.square(img_a)+np.square(img_b))
    sigma_c = np.std(chroma)

    # item-2
    img_lum = img_lum.flatten()
    sorted_index = np.argsort(img_lum)
    top_index = sorted_index[int(len(img_lum)*0.99)]
    bottom_index = sorted_index[int(len(img_lum)*0.01)]
    con_lum = img_lum[top_index] - img_lum[bottom_index]

    # item-3
    chroma = chroma.flatten()
    sat = np.divide(chroma, img_lum, out=np.zeros_like(chroma, dtype=np.float64), where=img_lum!=0)
    avg_sat = np.mean(sat)

    uciqe = sigma_c*coe_Metric[0] + con_lum*coe_Metric[1] + avg_sat*coe_Metric[2]
    return uciqe




In [ ]:



path1='E:/DataSet/LSUI/Test-L400/GT/'#要改
path2="E:/DataSet/LSUI/Test-L400/output/"#要改
path_list = os.listdir(path1)
path_list.sort(key=lambda x:int(x.split('.')[0]))
PSNR = []
SSIM = []
MSE=[]
UIQM=[]
UCIQE=[]

for item in path_list:
    impath1=path1+item
    impath2=path2+item
    # print(impath2)
    imgx= cv2.imread(impath1)
    # print(impath1)
    imgx=cv2.resize(imgx,(256,256))
      
    imgy= cv2.imread(impath2)
    if imgy is None:
        print("Error: Could not load image. Please check the file path.")
    imgy=cv2.resize(imgy,(256,256))

    
    #print(imgx.shape)
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
    
    psnr=(psnr1+psnr2+psnr3)/3.0

    
    mse=compute_mse(imgx,imgy)
    
    uiqm=calculate_uiqm(imgy)
    
    # uciqe=calculate_uciqe(imgy)
    uciqe = getUCIQE(imgy)
    
    print(impath1)
    
    print(psnr)

    print(mse)
    print(uiqm)
    print(uciqe)
    


    PSNR.append(psnr)
    
    MSE.append(mse)
    
    UIQM.append(uiqm)
    
    UCIQE.append(uciqe)
    


In [ ]:
PSNR=np.array(PSNR)    
print("PSNR:",PSNR.mean())


In [ ]:
UIQM=np.array(UIQM)    
print("UIQM:",UIQM.mean())

In [ ]:
UCIQE=np.array(UCIQE)    
print("UCIQE:",UCIQE.mean())